In [1]:
import pandas as pd
import time 
from datetime import datetime
from finvizfinance.screener.overview import Overview
import warnings
warnings.filterwarnings("ignore")

today = datetime.today().strftime('%Y_%m_%d')

In [2]:
indexes = ['S&P 500', 'NASDAQ 100', 'DJIA', 'RUSSELL 2000']
tickers = []
for ind in indexes:
    filters_dict = {'Index' : ind}
    foverview = Overview()
    foverview.set_filter(filters_dict=filters_dict)
    df_overview = foverview.screener_view()
    temp_tickers = df_overview['Ticker'].to_list()
    tickers = tickers + temp_tickers

tickers = list(set(tickers))

In [7]:
import certifi
import json
from urllib.request import urlopen

def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

In [142]:
year_now = datetime.now().year
main_df = pd.DataFrame()

for ticks in tickers:


    url = (f"https://financialmodelingprep.com/api/v3/ratios/{ticks}?period=quarter&apikey=PHaANSXTwW2zC5hpGFO1uhe8EkPXgio7")
    data = get_jsonparsed_data(url)

    dict = {'ticker' : [],
                'year' : [],
                'period' : [],
                'roce' : []}
    
    for i in range(len(data)):
        symbol = data[i]['symbol']
        roce = data[i]['returnOnCapitalEmployed']
        period = data[i]['period']
        year =  data[i]['calendarYear']
    
        dict['ticker'].append(symbol)
        dict['year'].append(year)
        dict['period'].append(period)
        dict['roce'].append(roce)

    df = pd.DataFrame(dict)
    df = df.groupby(by=['ticker','year']).agg({'roce':'sum', 'period': 'count'})
    df['adjusted_roce'] = (df['roce'] / df['period'] ) * 4
    df.reset_index(inplace=True)
    df = df[(pd.to_numeric(df['year']) >= (year_now-10)) & (pd.to_numeric(df['year']) <= year_now)]
    transposed_df = df.pivot(index='ticker', columns='year', values='roce')
    transposed_df.reset_index(inplace=True)
    main_df = pd.concat([main_df, transposed_df])
    #time.sleep(0.01)

In [146]:
main_df.to_csv(f"stock_financials/ROCE_Ratios_{today}.csv")
main_df.to_csv(f"stock_financials/ROCE_Ratios_recent.csv")

In [15]:
url = ("https://financialmodelingprep.com/api/v3/stock-price-change/AAPL?apikey=PHaANSXTwW2zC5hpGFO1uhe8EkPXgio7")
data = get_jsonparsed_data(url)
data[0]['1Y']

36.69364

In [8]:
url = (f"https://financialmodelingprep.com/api/v3/ratios/AAPL?period=quarter&apikey=PHaANSXTwW2zC5hpGFO1uhe8EkPXgio7")
data = get_jsonparsed_data(url)

In [9]:
data[0]

{'symbol': 'AAPL',
 'date': '2024-06-29',
 'calendarYear': '2024',
 'period': 'Q3',
 'currentRatio': 0.9529796997508053,
 'quickRatio': 0.9061417370692275,
 'cashRatio': 0.19422749650519663,
 'daysOfSalesOutstanding': 45.29745736036467,
 'daysOfInventoryOutstanding': 12.036052842794854,
 'operatingCycle': 57.33351020315952,
 'daysOfPayablesOutstanding': 92.87967201023883,
 'cashConversionCycle': -35.546161807079315,
 'grossProfitMargin': 0.462571551814589,
 'operatingProfitMargin': 0.29555708406682446,
 'pretaxProfitMargin': 0.2972125394919384,
 'netProfitMargin': 0.25004371801298714,
 'effectiveTaxRate': 0.1587040087863811,
 'returnOnAssets': 0.06467799717742422,
 'returnOnEquity': 0.32152065719254064,
 'returnOnCapitalEmployed': 0.1267676060563634,
 'netIncomePerEBT': 0.8412959912136189,
 'ebtPerEbit': 1.005601136005049,
 'ebitPerRevenue': 0.29555708406682446,
 'debtRatio': 0.3054895480260063,
 'debtEquityRatio': 1.5186184565569347,
 'longTermDebtToCapitalization': 0.5637262596138753